In [5]:
import cv2
from deepface import DeepFace 
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
import numpy as np
import time
import psycopg2
from tensorflow import keras

# duygu dictionary'sinin tanımlanması -> ilk değerleri sıfır
dominantDict = {
  "happy": 0,
  "neutral": 0,
  "sad": 0
}

# analiz edilen duyguların sayısal değerlerini tutmak için value'lar.
happyValue = 0;
neutralValue = 0;
sadValue = 0;

#video path
video ="video3.mp4"

#video içe aktarma
cap = cv2.VideoCapture(video)

if cap.isOpened() == False:
    print("video aktarılamadı.")

while True:
    ret, frame = cap.read()

    if ret == True:
        time.sleep(0.01) # kullanmazsak çok hızlı akar
        
        
        result = DeepFace.analyze(frame,  actions=['emotion'], enforce_detection=False)
        lastValue = result.get('emotion', {}).get(result['dominant_emotion']) # duyguların sayısal değerleri için variable
        
        if str(result['dominant_emotion']) == "happy":
            dominantDict["happy"] = dominantDict["happy"] +1
            happyValue = happyValue + result.get('emotion', {}).get(result['dominant_emotion'])

            # kaç kere happy analiz edildiğini yazdırır.
            print("number of happy:", dominantDict["happy"])
            # duygunun toplam sayısal değerini yazdırır.
            print("total happy value:", happyValue)
            print("last happy value:", lastValue)
    
        elif str(result['dominant_emotion']) == "neutral":
            dominantDict["neutral"] = dominantDict["neutral"] +1
            neutralValue = neutralValue + result.get('emotion', {}).get(result['dominant_emotion'])

            print("number of neutral:", dominantDict["neutral"])
            print("total neutral value:", neutralValue)
            print("last neutral value:", lastValue)

        else:
            dominantDict["sad"] = dominantDict["sad"] +1
            sadValue = sadValue + result.get('emotion', {}).get(result['dominant_emotion'])
            print("number of sad:", dominantDict["sad"])
            print( "total sad value:", sadValue)
            print("last sad value:", lastValue)
   
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
        #print (faceCascade.empty())
        faces = faceCascade.detectMultiScale(gray,1.1,4)

        #draw a rectangle around the faces
        for(x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        font = cv2.FONT_HERSHEY_SIMPLEX
        
        # user putText() method for inserting text on video -> duygu için
        if( result['dominant_emotion']=='happy' or result['dominant_emotion']=='sad' or result['dominant_emotion']=='neutral'):
            cv2.putText(frame,
                   result['dominant_emotion'],
                   (0, 50), font, 2,   #(0, 50) -> yazının konumu
                   (0, 0, 255), 2,  # color = red
                   cv2.LINE_4)

        # user putText() method for inserting text on video -> sayısal değer için
            cv2.putText(frame,
                   str(np.float32(lastValue)),
                   (0, 100), font, 1,
                   (0, 0, 255), 1,  # color = red
                   cv2.LINE_4)
        
        cv2.imshow("video", frame)
    
    else:break
    
    # tutulan duygu değerlerinin karşılaştırılarak en yükseğinin bulunması  

    finalResutReal = 0
    finalResultEmotion =""

    if np.float32(happyValue) > np.float32(neutralValue) and np.float32(happyValue) > np.float32(sadValue):
        print("dominant emotion: happy -> average: ", np.float32(happyValue) / np.float32(dominantDict["happy"]))
        # toplam happy sayısal değeri happy çıkma sayısına bölerek ortalama buluyoruz.
        finalResultReal = np.float32(happyValue) / np.float32(dominantDict["happy"])
        finalResultEmotion = "mutlu"

    elif np.float32(neutralValue) > np.float32(happyValue) and np.float32(neutralValue) > np.float32(sadValue):
        print("dominant emotion: neutral-> average: ", np.float32(neutralValue) / np.float32(dominantDict["neutral"]))
        finalResultReal = np.float32(neutralValue) / np.float32(dominantDict["neutral"])
        finalResultEmotion = "nötr"

    elif np.float32(sadValue) > np.float32(happyValue) and np.float32(sadValue) > np.float32(neutralValue):
        print("dominant emotion: sad-> average: ", np.float32(sadValue) / np.float32(dominantDict["sad"]))
        finalResultReal = np.float32(sadValue) / np.float32(dominantDict["sad"])
        finalResultEmotion = "üzgün"
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

print("\n",dominantDict) # duygular kaç kere ölçülmüş görmek için dict yazdırdık.
print("\n toplam mutluluk (sayısal değeri):", happyValue)
print("\n toplam nötrlük (sayısal değeri):", neutralValue)
print("\n toplam üzgünlük (sayısal değeri):", sadValue)

number of neutral: 1
total neutral value: 87.76732829006573
last neutral value: 87.76732829006573
dominant emotion: neutral-> average:  87.76733
number of neutral: 2
total neutral value: 147.0832958498802
last neutral value: 59.31596755981445
dominant emotion: neutral-> average:  73.54165
number of sad: 1
total sad value: 90.08185863494873
last sad value: 90.08185863494873
dominant emotion: neutral-> average:  73.54165
number of neutral: 3
total neutral value: 229.62400463016888
last neutral value: 82.5407087802887
dominant emotion: neutral-> average:  76.541336
number of neutral: 4
total neutral value: 319.3977430144157
last neutral value: 89.77373838424683
dominant emotion: neutral-> average:  79.849434
number of neutral: 5
total neutral value: 410.91355827244183
last neutral value: 91.51581525802612
dominant emotion: neutral-> average:  82.18271
number of neutral: 6
total neutral value: 494.48697116764447
last neutral value: 83.57341289520264
dominant emotion: neutral-> average:  82

dominant emotion: neutral-> average:  80.94508
number of sad: 26
total sad value: 2130.3286626396025
last sad value: 84.84046874528751
dominant emotion: neutral-> average:  80.94508
number of sad: 27
total sad value: 2200.495530611023
last sad value: 70.16686797142029
dominant emotion: neutral-> average:  80.94508
number of neutral: 35
total neutral value: 2848.1876243681822
last neutral value: 96.05474519077
dominant emotion: neutral-> average:  81.37679
number of sad: 28
total sad value: 2299.4104459342802
last sad value: 98.91491532325745
dominant emotion: neutral-> average:  81.37679
number of neutral: 36
total neutral value: 2899.4967509837065
last neutral value: 51.30912661552429
dominant emotion: neutral-> average:  80.54158
number of sad: 29
total sad value: 2373.3470184784464
last sad value: 73.93657254416615
dominant emotion: neutral-> average:  80.54158
number of sad: 30
total sad value: 2433.342222688728
last sad value: 59.99520421028137
dominant emotion: neutral-> average:

In [3]:
# veritabanı bağlantısı ve duygu sonucunun ilgili tabloya eklenmesi
from datetime import date
from datetime import datetime
import psycopg2

dbFinalResult = np.float64(finalResultReal) # postgreyle uyumlu olması için float64'e dönüştürdük.
analyseDate = datetime.today().strftime('%Y-%m-%d') # postgresql date formatına dönüştürme

try:
    connection = psycopg2.connect(user="postgres",
                                  password="1234",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="postgres")
    cursor = connection.cursor()

    postgres_insert_query = """ INSERT INTO duygu_veri_analizi (duygu_durumu_deger, duygu_durumu_kategori,birey_id,duygu_durumu_tarih) VALUES (%s,%s,%s,%s)"""
    record_to_insert = (dbFinalResult, finalResultEmotion, '12345678910',analyseDate)
    print("Inserted Values: ",finalResultEmotion ,"-", dbFinalResult)
    cursor.execute(postgres_insert_query, record_to_insert)

    connection.commit()
    count = cursor.rowcount
    print(count, "Record inserted successfully into table")

except (Exception, psycopg2.Error) as error:
    print("Failed to insert record into table", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

Inserted Values:  üzgün - 82.92723083496094
1 Record inserted successfully into table
PostgreSQL connection is closed
